In [6]:
import pandas as pd
import torch
import torch.nn as nn
import math

## Step 1: Convert text into embeddings

In [1]:
text = "Hello! How are you?"
print(text)

Hello! How you are ?


## Transformer Model Implementation

### Positional Encoding 

In [17]:
class PositionalEncoding(nn.Module):
    def __init__(self):
        super(PositionalEncoding, self).__init__() # skip the positionalEncoding inside super (after python 3)


    def forward(self, x):
        max_words, d_model = x.shape
        
        matrix = torch.zeros(max_words, d_model)
        two_i = 10000 ** (torch.arange(0, d_model, 2) / d_model)
        pos = torch.arange(0, max_words)

        matrix[:, 0::2] = torch.sin(pos / two_i)
        matrix[:, 1::2] = torch.cos(pos / two_i)
        
        x += matrix
        return x

### Self attention block

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self):
        super(SelfAttention, self).__init__()

    def forward(x);

        return x

In [ ]:
class MyTransformer(nn.Module):
    def __init__(self, max_words, d_model):
        super(MyTransformer, self).__init__()
        
        # embedding layer
        self.embedding_layer = nn.Embedding(max_words, d_model)

        # positional encoding
        self.positional_encoding = PositionalEncoding()

    def forward(self, x):
        x = self.embedding_layer(x)
        
        return x

In [16]:
matrix = torch.zeros(5, 6)
print(matrix)

# two_i = 10000 ** (torch.arange(0, 10, 2) / 10)
# print(two_i)

# pos = torch.arange(50)
# print(pos)

# print(torch.sin(pos / 10000))

a, b = matrix.shape
print(a, b)

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])
5 6
